In [1]:
import io

import pandas as pd
from Bio import Entrez, SeqIO
from mysql.connector import connection
from sshtunnel import SSHTunnelForwarder

# Укажите ваш email (обязательно для использования Entrez)
Entrez.email = "your.email@example.com"

In [2]:
with open("db_curated_server_info.txt", "r") as file:
    lines = file.readlines()

config = {}

for line in lines:
    line = line.strip()
    if line and not line.startswith("#"):
        key, value = line.split("=", 1)
        config[key] = value.strip()

server_name = config.get("server_name")
srever_port = int(config.get("srever_port"))
ssh_password = config.get("ssh_password")
ssh_username = config.get("ssh_username")
db_adress = config.get("db_adress")
db_port = int(config.get("db_port"))

In [3]:
tunnel = SSHTunnelForwarder(
    (server_name, srever_port),
    ssh_password=ssh_password,
    ssh_username=ssh_username,
    remote_bind_address=(db_adress, db_port),
)
tunnel.start()
print(tunnel.local_bind_port)

33495


In [4]:
conn = connection.MySQLConnection(
    user="db_user",
    password="db_password",
    host="localhost",
    port=tunnel.local_bind_port,
    database="db_name",
)
cursor = conn.cursor()

In [5]:
query = "SHOW TABLES;"
cursor.execute(query)
cursor.fetchall()

[('alternative_name',),
 ('histone',),
 ('histone_description',),
 ('histone_has_publication',),
 ('publication',),
 ('sequence',),
 ('sequence_has_publication',)]

In [6]:
add_histone = (
    "INSERT INTO histone "
    "(id, level, taxonomic_span, taxonomic_span_id, description, parent) "
    "VALUES (%(id)s, %(level)s, %(taxonomic_span)s, %(taxonomic_span_id)s, %(description)s, %(parent)s)"
)
add_histone_description = (
    "INSERT INTO histone_description "
    "(summary, taxonomy, genes, evolution, expression, knock_out, function, sequence, localization, deposition, structure, interactions, disease, caveats) "
    "VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
)
# add_publication = (
#     "INSERT INTO publication "
#     "(id, title, doi, author, year) "
#     "VALUES (%(id)s, %(title)s, %(doi)s, %(author)s, %(year)s)"
# )
# add_sequence = (
#     "INSERT INTO sequence "
#     "(accession, variant, gi, ncbi_gene_id, hgnc_gene_name, taxonomy_id, organism, phylum, class, taxonomy_group, info, sequence, variant_under_consideration) "
#     "VALUES (%(accession)s, %(variant)s, %(gi)s, %(ncbi_gene_id)s, %(hgnc_gene_name)s, %(taxonomy_id)s, %(organism)s, %(phylum)s, %(class)s, %(taxonomy_group)s, %(info)s, %(sequence)s, %(variant_under_consideration)s)"
# )
# add_sequence_has_publication = (
#     "INSERT INTO sequence_has_publication "
#     "(sequence_accession, publication_id) "
#     "VALUES (%s, %s)"
# )
add_alternate_names = (
    "INSERT INTO alternative_name "
    "(name, taxonomy, gene, splice, histone) "
    "VALUES (%(name)s, %(taxonomy)s, %(gene)s, %(splice)s, %(histone)s)"
)
# add_histone_has_publication = (
#     "INSERT INTO histone_has_publication "
#     "(histone_id, publication_id) "
#     "VALUES (%s, %s)"
# )

In [ ]:
data_histone_description = {
    "summary": None,
    "taxonomy": None,
    "genes": None,
    "evolution": None, 
    "expression": None,
    "knock_out": None,
    "function": None,
    "sequence": None,
    "localization": None,
    "deposition": None,
    "structure": None,
    "interactions": None,
    "disease": None,
    "caveats": None,
}

# To Do H1

## DEPRECATED <span style="color:red">Rename cH1 variants to cH1_(Mammalia?)</span>

## <span style="color:green">Update summary of cH1_(Mammalia?)</span>

### <span style="color:green">Update summary</span>

```cH1_(Mammalia?) is a group of replication dependent linker histones located within large histone gene clusters in mammals [happel_histone_2009, talbert_histone_2021].```

## <span style="color:green">Change parent of H1.1-5 variants to cH1</span>

## <span style="color:green">Rename H1.1-5 variants to cH1.1-5</span>

## <span style="color:green">Update summary of cH1.1-5 variants</span>

### <span style="color:green">Update summary</span>

```cH1.X is a replication dependent linker histone located within large histone gene clusters [happel_histone_2009, talbert_histone_2021].```

## <span style="color:green">Change taxonomic span of cH1.1-5 variants to Mammalia</span>

## <span style="color:green">Change parent of TS_H1.6 variant to cH1</span>

## <span style="color:green">Rename TS_H1.6 variant to cH1.6</span>

## <span style="color:green">Add alternate name TS_H1.6 of cH1.6</span>


# DONE

## DEPRECATED <span style="color:red">Rename cH1 variants to cH1_(Mammalia?)</span>

In [7]:
query = (
    "SELECT * FROM histone h LEFT JOIN histone_description hd "
    "ON h.description = hd.id "
    "WHERE h.id='cH1'"
)
cursor.execute(query)
pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])

,id,level,taxonomic_span,taxonomic_span_id,description,parent,id,summary,taxonomy,genes,...,knock_out,function,sequence,localization,deposition,structure,interactions,disease,caveats,relations
0,cH1,variant_group,null,null,8,H1,8,null,null,null,...,null,null,null,null,null,null,null,null,null,None


In [9]:
query = f"UPDATE histone SET id='cH1_(Mammalia?)' WHERE id='cH1'"
print(query)
cursor.execute(query)

UPDATE histone SET id='cH1_(Mammalia?)' WHERE id='cH1'


In [10]:
query = (
    "SELECT * FROM histone h LEFT JOIN histone_description hd "
    "ON h.description = hd.id "
    "WHERE h.id='cH1'"
)
cursor.execute(query)
pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])

,id,level,taxonomic_span,taxonomic_span_id,description,parent,id,summary,taxonomy,genes,...,knock_out,function,sequence,localization,deposition,structure,interactions,disease,caveats,relations


In [11]:
query = (
    "SELECT * FROM histone h LEFT JOIN histone_description hd "
    "ON h.description = hd.id "
    "WHERE h.id='cH1_(Mammalia?)'"
)
cursor.execute(query)
pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])

,id,level,taxonomic_span,taxonomic_span_id,description,parent,id,summary,taxonomy,genes,...,knock_out,function,sequence,localization,deposition,structure,interactions,disease,caveats,relations
0,cH1_(Mammalia?),variant_group,null,null,8,H1,8,null,null,null,...,null,null,null,null,null,null,null,null,null,None


In [12]:
# Make sure data is committed to the database
conn.commit()

## <span style="color:black">Update summary of cH1_(Mammalia?)</span>

### <span style="color:black">Update summary</span>

```cH1_(Mammalia?) is a group of replication dependent linker histones located within large histone gene clusters in mammals [happel_histone_2009, talbert_histone_2021].```

In [27]:
query = (
    "SELECT * FROM histone h LEFT JOIN histone_description hd "
    "ON h.description = hd.id "
    "WHERE h.id='cH1_(Mammalia?)'"
)
cursor.execute(query)
pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])

,id,level,taxonomic_span,taxonomic_span_id,description,parent,id,summary,taxonomy,genes,...,knock_out,function,sequence,localization,deposition,structure,interactions,disease,caveats,relations
0,cH1_(Mammalia?),variant_group,null,null,8,H1,8,cH1_(Mammalia?) is a group of replication depe...,null,null,...,null,null,null,null,null,null,null,null,null,None


In [28]:
summary_desc = "cH1 is a group of replication dependent linker histones located within large histone gene clusters in mammals [happel_histone_2009, talbert_histone_2021]."
query = f"UPDATE histone_description SET summary='{summary_desc}' WHERE id=8"
print(query)
cursor.execute(query)

UPDATE histone_description SET summary='cH1 is a group of replication dependent linker histones located within large histone gene clusters in mammals [happel_histone_2009, talbert_histone_2021].' WHERE id=8


In [31]:
query = f"UPDATE histone SET id='cH1' WHERE id='cH1_(Mammalia?)'"
print(query)
cursor.execute(query)

UPDATE histone SET id='cH1' WHERE id='cH1_(Mammalia?)'


In [33]:
query = (
    "SELECT * FROM histone h LEFT JOIN histone_description hd "
    "ON h.description = hd.id "
    "WHERE h.id='cH1'"
)
cursor.execute(query)
pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])

,id,level,taxonomic_span,taxonomic_span_id,description,parent,id,summary,taxonomy,genes,...,knock_out,function,sequence,localization,deposition,structure,interactions,disease,caveats,relations
0,cH1,variant_group,null,null,8,H1,8,cH1 is a group of replication dependent linker...,null,null,...,null,null,null,null,null,null,null,null,null,None


In [34]:
# Make sure data is committed to the database
conn.commit()

## <span style="color:black">Change parent of H1.1-5 variants to cH1</span>

In [21]:
variants = ["H1.1", "H1.2", "H1.3", "H1.4", "H1.5"]

In [35]:
query = (
    "SELECT * FROM histone "
)
cursor.execute(query)
df = pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])
df[df["id"].isin(variants)]

,id,level,taxonomic_span,taxonomic_span_id,description,parent
115,H1.1,variant_group,Homo,9605,11.0,H1
119,H1.2,variant_group,Homo,9605,13.0,H1
121,H1.3,variant_group,Homo,9605,14.0,H1
123,H1.4,variant_group,Homo,9605,15.0,H1
125,H1.5,variant_group,Homo,9605,16.0,H1


In [38]:
for v in variants:
    query = f"UPDATE histone SET parent='cH1', level='variant' WHERE id='{v}'"
    print(query)
    cursor.execute(query)

UPDATE histone SET parent='cH1', level='variant' WHERE id='H1.1'
UPDATE histone SET parent='cH1', level='variant' WHERE id='H1.2'
UPDATE histone SET parent='cH1', level='variant' WHERE id='H1.3'
UPDATE histone SET parent='cH1', level='variant' WHERE id='H1.4'
UPDATE histone SET parent='cH1', level='variant' WHERE id='H1.5'


In [39]:
query = (
    "SELECT * FROM histone "
)
cursor.execute(query)
df = pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])
df[df["id"].isin(variants)]

,id,level,taxonomic_span,taxonomic_span_id,description,parent
115,H1.1,variant,Homo,9605,11.0,cH1
119,H1.2,variant,Homo,9605,13.0,cH1
121,H1.3,variant,Homo,9605,14.0,cH1
123,H1.4,variant,Homo,9605,15.0,cH1
125,H1.5,variant,Homo,9605,16.0,cH1


## <span style="color:black">Rename H1.1-5 variants to cH1.1-5</span>

In [43]:
query = (
    "SELECT * FROM histone "
)
cursor.execute(query)
df = pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])
df[df["parent"].isin(variants)]

,id,level,taxonomic_span,taxonomic_span_id,description,parent
118,H1.1_(Homo_sapiens),variant,Homo sapiens,9606,61.0,H1.1
120,H1.2_(Homo_sapiens),variant,Homo sapiens,9606,63.0,H1.2
122,H1.3_(Homo_sapiens),variant,Homo sapiens,9606,64.0,H1.3
124,H1.4_(Homo_sapiens),variant,Homo sapiens,9606,65.0,H1.4
126,H1.5_(Homo_sapiens),variant,Homo sapiens,9606,66.0,H1.5


In [45]:
for v in variants:
    query = f"UPDATE histone SET parent=null WHERE parent='{v}'"
    print(query)
    cursor.execute(query)
    query = f"UPDATE histone SET id='c{v}' WHERE id='{v}'"
    print(query)
    cursor.execute(query)
    query = f"UPDATE histone SET id='c{v}_(Homo_sapiens)', parent='c{v}' WHERE id='{v}_(Homo_sapiens)'"
    print(query)
    cursor.execute(query)

UPDATE histone SET parent=null WHERE parent='H1.1'
UPDATE histone SET id='cH1.1' WHERE id='H1.1'
UPDATE histone SET id='cH1.1_(Homo_sapiens)', parent='cH1.1' WHERE id='H1.1_(Homo_sapiens)'
UPDATE histone SET parent=null WHERE parent='H1.2'
UPDATE histone SET id='cH1.2' WHERE id='H1.2'
UPDATE histone SET id='cH1.2_(Homo_sapiens)', parent='cH1.2' WHERE id='H1.2_(Homo_sapiens)'
UPDATE histone SET parent=null WHERE parent='H1.3'
UPDATE histone SET id='cH1.3' WHERE id='H1.3'
UPDATE histone SET id='cH1.3_(Homo_sapiens)', parent='cH1.3' WHERE id='H1.3_(Homo_sapiens)'
UPDATE histone SET parent=null WHERE parent='H1.4'
UPDATE histone SET id='cH1.4' WHERE id='H1.4'
UPDATE histone SET id='cH1.4_(Homo_sapiens)', parent='cH1.4' WHERE id='H1.4_(Homo_sapiens)'
UPDATE histone SET parent=null WHERE parent='H1.5'
UPDATE histone SET id='cH1.5' WHERE id='H1.5'
UPDATE histone SET id='cH1.5_(Homo_sapiens)', parent='cH1.5' WHERE id='H1.5_(Homo_sapiens)'


In [46]:
query = (
    "SELECT * FROM histone "
)
cursor.execute(query)
df = pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])
df[df["id"].isin(variants)]

,id,level,taxonomic_span,taxonomic_span_id,description,parent


In [47]:
query = (
    "SELECT * FROM histone "
)
cursor.execute(query)
df = pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])
df[df["id"].isin([f"c{v}" for v in variants])]

,id,level,taxonomic_span,taxonomic_span_id,description,parent
29,cH1.1,variant,Homo,9605,11.0,cH1
31,cH1.2,variant,Homo,9605,13.0,cH1
33,cH1.3,variant,Homo,9605,14.0,cH1
35,cH1.4,variant,Homo,9605,15.0,cH1
37,cH1.5,variant,Homo,9605,16.0,cH1


In [49]:
query = (
    "SELECT * FROM histone "
)
cursor.execute(query)
df = pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])
df[df["parent"].isin([f"c{v}" for v in variants])]

,id,level,taxonomic_span,taxonomic_span_id,description,parent
30,cH1.1_(Homo_sapiens),variant,Homo sapiens,9606,61.0,cH1.1
32,cH1.2_(Homo_sapiens),variant,Homo sapiens,9606,63.0,cH1.2
34,cH1.3_(Homo_sapiens),variant,Homo sapiens,9606,64.0,cH1.3
36,cH1.4_(Homo_sapiens),variant,Homo sapiens,9606,65.0,cH1.4
38,cH1.5_(Homo_sapiens),variant,Homo sapiens,9606,66.0,cH1.5


In [50]:
# Make sure data is committed to the database
conn.commit()

## <span style="color:black">Update summary of cH1.1-5 variants</span>

### <span style="color:black">Update summary</span>

```cH1.X is a replication dependent linker histone located within large histone gene clusters [happel_histone_2009, talbert_histone_2021].```

In [58]:
query = (
    "SELECT * FROM histone "
    "WHERE id LIKE 'cH1._'"
)
cursor.execute(query)
df = pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])
df

,id,level,taxonomic_span,taxonomic_span_id,description,parent
0,cH1.1,variant,Homo,9605,11,cH1
1,cH1.2,variant,Homo,9605,13,cH1
2,cH1.3,variant,Homo,9605,14,cH1
3,cH1.4,variant,Homo,9605,15,cH1
4,cH1.5,variant,Homo,9605,16,cH1


In [63]:
for i, row in df.iterrows():
    summary_desc = f"{row['id']} is a replication dependent linker histone located within large histone gene clusters in mammals [happel_histone_2009, talbert_histone_2021]."
    query = f"UPDATE histone_description SET summary='{summary_desc}' WHERE id={row['description']}"
    print(query)
    cursor.execute(query)

UPDATE histone_description SET summary='cH1.1 is a replication dependent linker histone located within large histone gene clusters in mammals [happel_histone_2009, talbert_histone_2021].' WHERE id=11
UPDATE histone_description SET summary='cH1.2 is a replication dependent linker histone located within large histone gene clusters in mammals [happel_histone_2009, talbert_histone_2021].' WHERE id=13
UPDATE histone_description SET summary='cH1.3 is a replication dependent linker histone located within large histone gene clusters in mammals [happel_histone_2009, talbert_histone_2021].' WHERE id=14
UPDATE histone_description SET summary='cH1.4 is a replication dependent linker histone located within large histone gene clusters in mammals [happel_histone_2009, talbert_histone_2021].' WHERE id=15
UPDATE histone_description SET summary='cH1.5 is a replication dependent linker histone located within large histone gene clusters in mammals [happel_histone_2009, talbert_histone_2021].' WHERE id=16


In [64]:
query = (
    "SELECT * FROM histone h LEFT JOIN histone_description hd "
    "ON h.description = hd.id "
    "WHERE h.id LIKE 'cH1._'"
)
cursor.execute(query)
pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])

,id,level,taxonomic_span,taxonomic_span_id,description,parent,id,summary,taxonomy,genes,...,knock_out,function,sequence,localization,deposition,structure,interactions,disease,caveats,relations
0,cH1.1,variant,Homo,9605,11,cH1,11,cH1.1 is a replication dependent linker histon...,null,null,...,null,null,null,null,null,null,null,null,null,None
1,cH1.2,variant,Homo,9605,13,cH1,13,cH1.2 is a replication dependent linker histon...,null,null,...,null,null,null,null,null,null,null,null,null,None
2,cH1.3,variant,Homo,9605,14,cH1,14,cH1.3 is a replication dependent linker histon...,null,null,...,null,null,null,null,null,null,null,null,null,None
3,cH1.4,variant,Homo,9605,15,cH1,15,cH1.4 is a replication dependent linker histon...,null,null,...,null,null,null,null,null,null,null,null,null,None
4,cH1.5,variant,Homo,9605,16,cH1,16,cH1.5 is a replication dependent linker histon...,null,null,...,null,null,null,null,null,null,null,null,null,None


In [65]:
# Make sure data is committed to the database
conn.commit()

## <span style="color:black">Change taxonomic span of cH1.1-5 variants to Mammalia</span>

In [67]:
variants = ["cH1.1", "cH1.2", "cH1.3", "cH1.4", "cH1.5"]

In [68]:
query = (
    "SELECT * FROM histone "
    "WHERE id LIKE 'cH1._'"
)
cursor.execute(query)
pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])

,id,level,taxonomic_span,taxonomic_span_id,description,parent
0,cH1.1,variant,Homo,9605,11,cH1
1,cH1.2,variant,Homo,9605,13,cH1
2,cH1.3,variant,Homo,9605,14,cH1
3,cH1.4,variant,Homo,9605,15,cH1
4,cH1.5,variant,Homo,9605,16,cH1


In [70]:
for v in variants:
    query = f"UPDATE histone SET taxonomic_span='Mammalia', taxonomic_span_id='40674' WHERE id='{v}'"
    print(query)
    cursor.execute(query)

UPDATE histone SET taxonomic_span='Mammalia', taxonomic_span_id='40674' WHERE id='cH1.1'
UPDATE histone SET taxonomic_span='Mammalia', taxonomic_span_id='40674' WHERE id='cH1.2'
UPDATE histone SET taxonomic_span='Mammalia', taxonomic_span_id='40674' WHERE id='cH1.3'
UPDATE histone SET taxonomic_span='Mammalia', taxonomic_span_id='40674' WHERE id='cH1.4'
UPDATE histone SET taxonomic_span='Mammalia', taxonomic_span_id='40674' WHERE id='cH1.5'


In [71]:
query = (
    "SELECT * FROM histone "
    "WHERE id LIKE 'cH1._'"
)
cursor.execute(query)
pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])

,id,level,taxonomic_span,taxonomic_span_id,description,parent
0,cH1.1,variant,Mammalia,40674,11,cH1
1,cH1.2,variant,Mammalia,40674,13,cH1
2,cH1.3,variant,Mammalia,40674,14,cH1
3,cH1.4,variant,Mammalia,40674,15,cH1
4,cH1.5,variant,Mammalia,40674,16,cH1


In [72]:
# Make sure data is committed to the database
conn.commit()

## <span style="color:black">Change parent of TS_H1.6 variant to cH1</span>

In [73]:
query = (
    "SELECT * FROM histone "
    "WHERE id='TS_H1.6'"
)
cursor.execute(query)
pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])

,id,level,taxonomic_span,taxonomic_span_id,description,parent
0,TS_H1.6,variant_group,Mammalia,40674,19,H1


In [75]:
query = f"UPDATE histone SET parent='cH1' WHERE id='TS_H1.6'"
print(query)
cursor.execute(query)

UPDATE histone SET parent='cH1' WHERE id='TS_H1.6'


In [76]:
query = (
    "SELECT * FROM histone "
    "WHERE id='TS_H1.6'"
)
cursor.execute(query)
pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])

,id,level,taxonomic_span,taxonomic_span_id,description,parent
0,TS_H1.6,variant_group,Mammalia,40674,19,cH1


In [77]:
# Make sure data is committed to the database
conn.commit()

## <span style="color:black">Rename TS_H1.6 variant to cH1.6</span>

In [78]:
query = (
    "SELECT * FROM histone "
)
cursor.execute(query)
df = pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])
df[df["parent"]=="TS_H1.6"]

,id,level,taxonomic_span,taxonomic_span_id,description,parent
127,H1.6_(Homo_sapiens),variant,Homo sapiens,9606,68.0,TS_H1.6


In [81]:
query = f"UPDATE histone SET parent=null WHERE parent='TS_H1.6'"
print(query)
cursor.execute(query)
query = f"UPDATE histone SET id='cH1.6' WHERE id='TS_H1.6'"
print(query)
cursor.execute(query)
query = f"UPDATE histone SET id='cH1.6_(Homo_sapiens)', parent='cH1.6' WHERE id='H1.6_(Homo_sapiens)'"
print(query)
cursor.execute(query)

UPDATE histone SET parent=null WHERE parent='TS_H1.6'
UPDATE histone SET id='cH1.6' WHERE id='TS_H1.6'
UPDATE histone SET id='cH1.6_(Homo_sapiens)', parent='cH1.6' WHERE id='H1.6_(Homo_sapiens)'


In [82]:
query = (
    "SELECT * FROM histone "
)
cursor.execute(query)
df = pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])
df[df["parent"]=="TS_H1.6"]

,id,level,taxonomic_span,taxonomic_span_id,description,parent


In [84]:
query = (
    "SELECT * FROM histone "
)
cursor.execute(query)
df = pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])
df[df["parent"]=="cH1.6"]

,id,level,taxonomic_span,taxonomic_span_id,description,parent
40,cH1.6_(Homo_sapiens),variant,Homo sapiens,9606,68.0,cH1.6


In [85]:
# Make sure data is committed to the database
conn.commit()

## <span style="color:black">Add alternate name TS_H1.6 of cH1.6</span>

In [86]:
query = "SELECT * FROM alternative_name " "WHERE histone='cH1.6'"
cursor.execute(query)
pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])

,id,name,taxonomy,gene,splice,histone
0,8,H1t,None,None,None,cH1.6


In [87]:
data_alternate_name = {
    "name": "TS_H1.6",
    "taxonomy": None,
    "gene": None,
    "splice": None,
    "histone": "cH1.6",
}
cursor.execute(add_alternate_names, data_alternate_name)

In [88]:
query = "SELECT * FROM alternative_name " "WHERE histone='cH1.6'"
cursor.execute(query)
pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])

,id,name,taxonomy,gene,splice,histone
0,8,H1t,None,None,None,cH1.6
1,123,TS_H1.6,None,None,None,cH1.6


In [89]:
# Make sure data is committed to the database
conn.commit()

# Close connections

In [90]:
cursor.close()
conn.close()
tunnel.stop()